In [2]:
!pip -q install -U transformers datasets accelerate scikit-learn pandas

TRAIN_CSV = "/content/iemocap6_emoberta_train.csv"
VAL_CSV   = "/content/iemocap6_emoberta_val.csv"
TEST_CSV  = "/content/iemocap6_emoberta_test.csv"


In [ ]:

import numpy as np, pandas as pd, torch, os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, set_seed
from sklearn.metrics import accuracy_score, f1_score


# ====== CONFIG ======
MODEL_BASE = "bert-base-uncased"
TEXT_COL = "Utterance"
LABEL_COL = "Emotion"

IEMO6 = ["neutral","frustration","sadness","anger","excited","happiness"]
label2id = {l:i for i,l in enumerate(IEMO6)}
id2label = {i:l for l,i in label2id.items()}

SEEDS = [42,43,44,45,46]

LR = 3e-5
EPOCHS = 5
LABEL_SMOOTHING = 0.1


BATCH_TRAIN = 16
BATCH_EVAL  = 32
MAX_LEN = 256
OUT_ROOT = "bert_iemocap6"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
tok = AutoTokenizer.from_pretrained(MODEL_BASE, use_fast=True)


In [ ]:
def load_df(path):
    df = pd.read_csv(path).dropna(subset=[TEXT_COL, LABEL_COL]).copy()
    df[TEXT_COL]  = df[TEXT_COL].astype(str)
    df[LABEL_COL] = df[LABEL_COL].astype(str).str.strip().str.lower()
    df = df[df[LABEL_COL].isin(IEMO6)].copy()
    return df

def to_ds(df):
    ds = Dataset.from_pandas(df[[TEXT_COL, LABEL_COL]], preserve_index=False)
    def enc(batch):
        out = tok(batch[TEXT_COL], truncation=True, padding=False, max_length=MAX_LEN)
        out["labels"] = [label2id[x] for x in batch[LABEL_COL]]
        return out
    return ds.map(enc, batched=True, remove_columns=[TEXT_COL, LABEL_COL])

train_ds = to_ds(load_df(TRAIN_CSV))
val_ds   = to_ds(load_df(VAL_CSV))
test_ds  = to_ds(load_df(TEST_CSV))

print("Counts:", len(train_ds), len(val_ds), len(test_ds))  # 4778 / 980 / 1622 expected

def compute_metrics(eval_pred):
    logits, y_true = eval_pred
    y_pred = np.argmax(logits, axis=1)
    return {
        "acc": accuracy_score(y_true, y_pred),
        "weighted_f1": f1_score(y_true, y_pred, average="weighted"),
        "macro_f1": f1_score(y_true, y_pred, average="macro"),
    }

In [5]:

rows = []
best_ckpts = {}  # seed -> checkpoint path

for seed in SEEDS:
    print("\n" + "="*20, "SEED", seed, "="*20)
    set_seed(seed)

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_BASE,
        num_labels=len(IEMO6),
        label2id=label2id,
        id2label=id2label
    ).to(DEVICE)

    args = TrainingArguments(
        output_dir=f"{OUT_ROOT}_seed{seed}",
        eval_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="weighted_f1",
        greater_is_better=True,

        learning_rate=LR,
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_TRAIN,
        per_device_eval_batch_size=BATCH_EVAL,
        weight_decay=0.01,
        warmup_ratio=0.06,

        label_smoothing_factor=LABEL_SMOOTHING,  # <-- change
        fp16=torch.cuda.is_available(),
        report_to="none",
        seed=seed,
        logging_steps=50,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tok,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    best_ckpts[seed] = trainer.state.best_model_checkpoint
    res = trainer.evaluate(test_ds)

    rows.append({
        "seed": seed,
        "test_acc": float(res["eval_acc"]),
        "test_weighted_f1": float(res["eval_weighted_f1"]),
        "test_macro_f1": float(res["eval_macro_f1"]),
        "best_ckpt": best_ckpts[seed],
    })

df = pd.DataFrame(rows)
print("\nPer-seed results:")
display(df)

print("\nMEAN:")
display(df.drop(columns=["seed","best_ckpt"]).mean().to_frame("mean"))

print("\nSTD:")
display(df.drop(columns=["seed","best_ckpt"]).std().to_frame("std"))


Map:   0%|          | 0/4778 [00:00<?, ? examples/s]

Map:   0%|          | 0/980 [00:00<?, ? examples/s]

Map:   0%|          | 0/1622 [00:00<?, ? examples/s]

Counts: 4778 980 1622

==================== SEED 42 ====================


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3667648277.py:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


...

,seed,test_acc,test_weighted_f1,test_macro_f1,best_ckpt
0,42,0.560419,0.558167,0.541635,bert_iemocap6_seed42/checkpoint-1196
1,43,0.541924,0.536208,0.512484,bert_iemocap6_seed43/checkpoint-598
2,44,0.549938,0.547965,0.531175,bert_iemocap6_seed44/checkpoint-1196
3,45,0.554254,0.550837,0.531940,bert_iemocap6_seed45/checkpoint-1196
4,46,0.564735,0.561003,0.541296,bert_iemocap6_seed46/checkpoint-897



MEAN:


,mean
test_acc,0.554254
test_weighted_f1,0.550836
test_macro_f1,0.531706



STD:


,std
test_acc,0.008924
test_weighted_f1,0.009738
test_macro_f1,0.011836


In [6]:
import os, shutil

for seed, ckpt_path in best_ckpts.items():
    best_dir = f"{OUT_ROOT}_seed{seed}_BEST"
    if os.path.exists(best_dir):
        shutil.rmtree(best_dir)
    shutil.copytree(ckpt_path, best_dir)
    tok.save_pretrained(best_dir)
    print(f"Saved BEST for seed {seed}: {best_dir}")


Saved BEST for seed 42: bert_iemocap6_seed42_BEST
Saved BEST for seed 43: bert_iemocap6_seed43_BEST
Saved BEST for seed 44: bert_iemocap6_seed44_BEST
Saved BEST for seed 45: bert_iemocap6_seed45_BEST
Saved BEST for seed 46: bert_iemocap6_seed46_BEST
